## 1. Imports

In [1]:
import os
os.chdir("..")

In [1]:
import cv2
import pandas as pd
from typing import List

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

## 2. Dataset - Images and Labels

In [2]:
images_dir =  os.path.join("data", "bdd100k_images_100k", "bdd100k", "images", "100k")
labels_dir = os.path.join("data", "bdd100k_labels_csv",)

train_images = os.listdir(os.path.join(images_dir, "train"))
val_images = os.listdir(os.path.join(images_dir, "val"))

train_label_df = pd.read_csv(os.path.join(labels_dir, "train_labels.csv"))
val_label_df = pd.read_csv(os.path.join(labels_dir, "val_labels.csv"))

train_labels = train_label_df["file_name"].unique().tolist()
val_labels = val_label_df["file_name"].unique().tolist()

train_files = set(train_images).intersection(set(train_labels))
val_files = set(val_images).intersection(set(val_labels))

print(f"Total Train Images: {len(train_files)}")
print(f"Total Val Images: {len(val_files)}")

print(f"Total Train Labels: {len(train_labels)}")
print(f"Total Val Labels: {len(val_labels)}")

Total Train Images: 69863
Total Val Images: 10000
Total Train Labels: 69863
Total Val Labels: 10000


## 3. Helper class/ functions

In [3]:
def save_labels_txt(label_df: pd.DataFrame, attributes: List, file_names: List, save_dir: str):

    for attribute in attributes:
        os.makedirs(os.path.join(save_dir, attribute), exist_ok=True)
        
    if isinstance(label_df["bbox"].iloc[0], str):
        label_df = label_df.copy()
        label_df["bbox"] = label_df["bbox"].apply(eval)
        
    for file_name in tqdm(file_names, total=len(file_names)):
        
        scene_df = label_df.query("file_name == @file_name")
        if scene_df.empty:
            continue
        
        for attribute in attributes:

            if attribute == "all":
                target_df = scene_df
            elif attribute in ["occluded", "truncated", "small", "medium", "large"]:
                target_df = scene_df.query(f"{attribute} == True and uncertain == False")
            elif attribute == "uncertain":
                target_df = scene_df.query(f"{attribute} == True")

            if target_df.empty:
                continue

            save_file = os.path.join(save_dir, attribute, file_name.replace(".jpg", ".txt"))
            with open(save_file, "w") as f:
                for label, bbox in zip(target_df["label"], target_df["bbox"]):
                    f.write(f"{label} " + " ".join(map(str, bbox)) + "\n")

In [4]:
def get_mean_std(loader: DataLoader):
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for image in loader:
        channels_sum += torch.mean(image, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(image ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

class MeanStdDataset(Dataset):
    def __init__(self, image_paths, transform):
        
        self.image_paths = image_paths
        self.transform = transform
    
    def read_image(self, path):
        return cv2.cvtColor(
            cv2.imread(path, cv2.IMREAD_COLOR), 
            cv2.COLOR_BGR2RGB
        )

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        
        # Load image
        image = self.read_image(self.image_paths[idx])

        # transform image
        image = self.transform(image=image)["image"]

        return image

## 4. Compute Dataset Mean and Std (on Train data only)

In [ ]:
images_dir_train = os.path.join(images_dir, "train")
image_paths = [os.path.join(images_dir_train, path) for path in train_files]

IMAGE_HEIGHT = 480
IMAGE_WIDTH = 640
BATCH_SIZE = 16

mean_std_transforms = A.Compose([
    A.Resize(width=IMAGE_WIDTH, height=IMAGE_HEIGHT, p=1.0),
    A.Normalize(mean=[0, 0, 0], std=[1, 1, 1], max_pixel_value=255,),
    ToTensorV2(),
])

mean_std_dataset = MeanStdDataset(
    image_paths,
    transform=mean_std_transforms
)

mean_std_dataloader = DataLoader(mean_std_dataset, batch_size=BATCH_SIZE, shuffle=False)
mean, std = get_mean_std(mean_std_dataloader)
print(mean, std)

In [5]:
save_path = os.path.join("data", "mean_std_bdd100k.txt")

# write to file
with open(save_path, mode="w") as f:
    f.write(f"mean {[round(item.item(), 4) for item in mean]}\n")
    f.write(f"std {[round(item.item(), 4) for item in std]}\n")

## 5. Save Labels in Training format

### Lable format: label_name x1 y1 x2 y2

In [4]:
train_save_dir = os.path.join("data", "bdd100k_labels_txt", "bdd100k", "labels", "train")
val_save_dir = os.path.join("data", "bdd100k_labels_txt", "bdd100k", "labels", "val")

attributes = ["all", "occluded", "truncated", "small", "medium", "large", "uncertain"]
save_labels_txt(train_label_df, attributes, train_files, train_save_dir)
save_labels_txt(val_label_df, attributes, val_files, val_save_dir)

  0%|          | 0/69863 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]